In [1]:
import cv2
import numpy as np
import time
import os
import math
import glob

# Create directories for saving videos and pictures
if not os.path.exists('video'):
    os.makedirs('video')

if not os.path.exists('picture'):
    os.makedirs('picture')

if not os.path.exists('saved'):
    os.makedirs('saved')  # Directory for saving valid files

cap = cv2.VideoCapture(0)  # Adjust to your camera source
fourcc = cv2.VideoWriter_fourcc(*'XVID')

def calculate_speed(start_point, end_point, time_elapsed):
    distance = math.sqrt((end_point[0] - start_point[0]) ** 2 + 
                         (end_point[1] - start_point[1]) ** 2)
    speed = distance / time_elapsed
    return speed

def pixels_to_kmh(pixels_per_second, pixel_to_meter_ratio):
    meters_per_second = pixels_per_second * pixel_to_meter_ratio
    kmh = meters_per_second * 3.6
    return kmh

def delete_file(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

def move_saved_files():
    """Move valid files to a 'saved' directory to prevent accidental deletion."""
    for filename in glob.glob('picture/*.png'):
        os.rename(filename, os.path.join('saved', os.path.basename(filename)))

def update_status_window(status_text):
    """Updates the status text displayed in a separate window."""
    status_img = np.zeros((200, 640, 3), np.uint8)
    y0, dy = 20, 30
    for i, line in enumerate(status_text.split('\n')):
        y = y0 + i * dy
        cv2.putText(status_img, line, (10, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
    cv2.imshow("Status", status_img)

def motion_detection():
    screenshot_count = 0  
    max_screenshots = 5  
    video_recording = False 
    last_person_detected_time = None  
    video_filename = None  
    out = None  
    max_frames = 30  
    frame_count = 0  
    start_point = None  
    end_point = None  

    pixel_to_meter_ratio = 0.0002645833  
    ret, frame1 = cap.read()
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (5, 5), 0)

    status_text = "Idle..."

    while True:
        ret, frame2 = cap.read()
        if not ret:
            status_text = "Error reading frame."
            update_status_window(status_text)
            break

        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.GaussianBlur(gray2, (5, 5), 0)

        delta = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(delta, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        person_detected = False

        for contour in contours:
            if cv2.contourArea(contour) < 500:
                continue

            cv2.drawContours(frame2, [contour], -1, (0, 255, 0), 2)
            person_detected = True

            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                end_point = (cX, cY)  

                if start_point is None:
                    start_point = (cX, cY)  

        if person_detected:
            if not video_recording:
                video_recording = True
                status_text = "Recording started..."
                screenshot_count = 0

                timestamp = time.strftime("%Y%m%d-%H%M%S")
                video_filename = os.path.join('video', f"motion_{timestamp}.avi")
                out = cv2.VideoWriter(video_filename, fourcc, 20.0, (640, 480))

            out.write(frame2)

            if screenshot_count < max_screenshots:
                screenshot_filename = os.path.join('picture', f"screenshot_{int(time.time())}.png")
                cv2.imwrite(screenshot_filename, frame2)
                status_text += f"\nScreenshot taken: {screenshot_filename}"
                screenshot_count += 1

            last_person_detected_time = time.time()
            frame_count += 1  

        else:
            if last_person_detected_time and time.time() - last_person_detected_time > 0.5:
                status_text = "No movement detected for 0.5 seconds. Stopping video recording."
                video_recording = False
                last_person_detected_time = None
                out.release()  

                if start_point is not None and end_point is not None:
                    time_elapsed = frame_count / 20  
                    speed_pixels_per_second = calculate_speed(start_point, end_point, time_elapsed)
                    speed_kmh = pixels_to_kmh(speed_pixels_per_second, pixel_to_meter_ratio)  
                    status_text += f"\nObject speed: {speed_pixels_per_second:.2f} pixels/sec, {speed_kmh:.2f} km/h"

                    if speed_kmh >= 20:
                        status_text += f"\nObject exceeded 20 km/h: {speed_kmh:.2f} km/h. Saving files."
                        move_saved_files()  # Move screenshots to prevent deletion
                    else:
                        status_text += f"\nObject below 20 km/h: {speed_kmh:.2f} km/h. Deleting files."
                        delete_file(video_filename)
                        for filename in glob.glob('picture/*.png'):
                            delete_file(filename)

                start_point = None
                end_point = None
                frame_count = 0

        if video_recording:
            cv2.putText(frame2, "Recording...", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
            cv2.putText(frame2, "Idle", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        cv2.imshow("Motion Detection", frame2)
        update_status_window(status_text)  # Update the status window
        gray1 = gray2.copy()

        if cv2.waitKey(1) == 27:
            break

    if out is not None:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    motion_detection()


KeyboardInterrupt: 